In [1]:
%reset -f

In [ ]:
from sklearn.model_selection import train_test_split

In [2]:
import openpyxl
from openpyxl.utils import column_index_from_string
import pandas as pd
import importlib
import simulation_class  

importlib.reload(simulation_class)
HeatPumpSimulator = simulation_class.HeatPumpSimulator


## Extraction of the Output Data

Extraction of output data from the `All data` sheet of the Excel file and storing in a DataFrame `df_output`.

- Column K : Water out -> water temperature leaving the heat pump; FMU output
- Column O : Power out -> thermal power of the heat pump (hot side); FMU output
- Column P : Power in -> electrical power of the heat pump; FMU output
- Column Q : COP -> coefficient of performance of the heat pump; FMU output

In [3]:
excel_file_path = './../data/Databrut.xlsx'

workbook = openpyxl.load_workbook(excel_file_path)
sheet = workbook.active  

# Define the columns and corresponding names
columns_info = {
    "K": "Water_out",
    "O": "Power_out",
    "P": "Power_in",
    "Q": "COP"
}

# Create a dictionary to store the selected columns
stored_data = {name: [] for name in columns_info.values()}

# Iterate over the columns and store data in the dictionary
for col_letter, col_name in columns_info.items():
    col_index = column_index_from_string(col_letter)
    if col_index <= sheet.max_column:
        for cell in sheet[col_letter]:
            stored_data[col_name].append(cell.value)
    else:
        print(f"Column {col_letter} not found in the sheet.")

workbook.close()



In [4]:
df = pd.DataFrame(stored_data)

df_output = df.apply(pd.to_numeric, errors='coerce').dropna()

# Conversion de la température de l'eau de °C à K
df_output['Water_out'] = df_output['Water_out'] + 273.15

#Renommer les  colonnes pour correspondre aux noms des sorties FMU
df_output = df_output.rename(columns={'Power_out': 'y_heatPower', 'Power_in': 'y_elecPower', 'COP': 'y_COP'})

df_output.head()


,Water_out,y_heatPower,y_elecPower,y_COP
4,308.248729,8628.775284,2022.298343,4.279641
5,308.320939,8589.444308,2044.149171,4.214619
6,308.179724,7641.797307,1748.723757,4.383122
7,308.268453,6873.783742,1477.259669,4.667061
8,308.200442,6035.440249,1248.497238,4.848746


In [5]:
#Export l'output en fichier Excel
Outputpath = './../data/Outputdata.xlsx'
df_output.to_excel(Outputpath, index=False)

## Extraction of the Input Data

Extraction of the input data from the `All data` sheet of the Excel file and storing in a DataFrame `df_input`.

- Column I : Frequency -> compressor frequency; FMU input
- Column J : Water in -> water temperature at the heat pump inlet; FMU input
- Column L : Flow -> water flow; FMU input
- Column M : Defrosts -> indicates if defrosting took place during the test

In [6]:
excel_file_path = './../data/Databrut.xlsx'

workbook = openpyxl.load_workbook(excel_file_path)
sheet = workbook.active

columns_info = {
    "I": "Frequency",
    "J": "Water_in",
    "L": "Flow",
    "M": "Defrosts"  
}

stored_data = {name: [] for name in columns_info.values()}

for col_letter, col_name in columns_info.items():
    col_index = column_index_from_string(col_letter)
    if col_index <= sheet.max_column:
        for row in sheet.iter_rows(min_row=2, max_col=col_index, min_col=col_index, values_only=True):  
            stored_data[col_name].extend(row)
    else:
        print(f"Column {col_letter} not found in the sheet.")

workbook.close()

df_input = pd.DataFrame(stored_data)

for column in ["Frequency", "Water_in", "Flow"]:
    df_input[column] = pd.to_numeric(df_input[column], errors='coerce')

df_input.dropna(subset=["Frequency", "Water_in", "Flow"], inplace=True)


# Création de la colonne 'T_air' avec les valeurs correctes
NbA12W35,NbA12W45 = 9,9
NbA7W35,NbA7W45,NbA7W55 = 13,12,4
NbA2W35,NbA2W45,NbA2W55  = 5,5,5
NbAmoins7W35,NbAmoins7W45,NbAmoins7W55 = 5,5,5

NbA12 = NbA12W35 + NbA12W45
NbA7 = NbA7W35 + NbA7W45 + NbA7W55
NbA2 = NbA2W35 + NbA2W45 + NbA2W55
NbAmoins7 = NbAmoins7W35 + NbAmoins7W45 + NbAmoins7W55

values = [12]*NbA12 + [7]*NbA7 + [2]*NbA2 + [-7]*NbAmoins7

if len(values) == len(df_input):
    df_input['T_air'] = values
else:
    print("Erreur : La longueur des valeurs de 'T_air' ne correspond pas au DataFrame.")

columns_order = ["Frequency", "Water_in", "Flow", "T_air", "Defrosts"] 
df_input = df_input[columns_order]  # Réorganisation des colonnes

# df_input.head()


In [7]:
# Conversion de la température de l'eau de °C à K
df_input['Water_in'] = df_input['Water_in'] + 273.15

# Conversion du débit d'eau de m³/h à m³/s
df_input['Flow'] = df_input['Flow'] / 3600

# Conversion de la température de l'air de °C à K
df_input['T_air'] = df_input['T_air'] + 273.15

df_input.head()

,Frequency,Water_in,Flow,T_air,Defrosts
3,105.6,303.262818,0.000415,285.15,no
4,98.4,303.289006,0.000409,285.15,no
5,88.2,303.231215,0.000370,285.15,no
6,78.0,303.359724,0.000336,285.15,no
7,68.4,303.246077,0.000292,285.15,no


In [8]:
# export l'input en fichier Excel
Inputpath = './../data/Inputdata.xlsx'
df_input.to_excel(Inputpath, index=False)


## Calibration (defrost = no) 

Firstly, we ignore the tests where "Defrosts" = yes (column M)

In [9]:
df_no_defrosts = df_input[df_input['Defrosts'] == 'no']

In this notebook, we are importing the Python class `HeatPumpSimulator` that utilizes FMpy to simulate FMU. This class is responsible for performing the simulation of the heat pump system. Below are the defined inputs and outputs for the simulation.


#### FMU Inputs

- **u_compressorFrequency:** Compressor Frequency *Unit:* Hz

- **u_VFlow:** Water Flow *Unit:* m³/s

- **u_T_water:** Water Temperature Entering the Heat Pump *Unit:* K

- **u_T_air:** Air Temperature at the Heat Pump Inlet *Unit:* K

#### FMU Outputs

- **y_T_out:** Water Temperature Leaving the Heat Pump *Unit:* K

- **y_heatPower:** Thermal Power of the Heat Pump (Hot Side) *Unit:* W

- **y_elecPower:** Electrical Power of the Heat Pump *Unit:* W

- **y_COP:** Coefficient of Performance of the Heat Pump *Unit:* []


In [11]:
simulator = HeatPumpSimulator('../FMU/HPFMU_20_Linux.fmu')

last_rows = []

# Itération sur les 10 premières lignes du DataFrame d'entrée
for index, row in df_no_defrosts.head(10).iterrows():
    start_values = {
        'u_compressorFrequency': row['Frequency'],
        'u_VFlow': row['Flow'],
        'u_T_watrer_in': row['Water_in'],
        'u_T_air': row['T_air'],
        # Valeurs fixes des paramètres pour l'instant
        'x_areaLeakage': 5e-7,
        'x_areaSuctionValve': 5e-4,
        'x_areaDischargeValve': 5e-5,
        'x_relativeDeadSpace': 0.05,
        'x_driveEfficiency': 0.9
    }

    simulation_result = simulator.simulate(start_values)
    df_simulation_result = pd.DataFrame(simulation_result)
    if not df_simulation_result.empty:
        last_row = df_simulation_result.iloc[-1]
        last_rows.append(last_row)
    else:
        print("Simulation result dataframe is empty.")


Le temps d'exécution dépasse 3 minutes. Tentative d'arrêt du processus.
Simulation result dataframe is empty.
Le temps d'exécution dépasse 3 minutes. Tentative d'arrêt du processus.
Simulation result dataframe is empty.
Le temps d'exécution dépasse 3 minutes. Tentative d'arrêt du processus.
Simulation result dataframe is empty.
Le temps d'exécution dépasse 3 minutes. Tentative d'arrêt du processus.
Simulation result dataframe is empty.
Le temps d'exécution dépasse 3 minutes. Tentative d'arrêt du processus.
Simulation result dataframe is empty.
Le temps d'exécution dépasse 3 minutes. Tentative d'arrêt du processus.
Simulation result dataframe is empty.
Le temps d'exécution dépasse 3 minutes. Tentative d'arrêt du processus.
Simulation result dataframe is empty.
Le temps d'exécution dépasse 3 minutes. Tentative d'arrêt du processus.
Simulation result dataframe is empty.
Le temps d'exécution dépasse 3 minutes. Tentative d'arrêt du processus.
Simulation result dataframe is empty.
Le temps d

## Multi-opti

In [10]:
OUTPUT = {
    'Water_out' : df_output['Water_out'],
    'y_heatPower': df_output['y_heatPower'],  
    'y_elecPower': df_output['y_elecPower'],
    'y_COP': df_output['y_COP']
}

INPUT = {
    'u_compressorFrequency': df_no_defrosts['Frequency'],
    'u_VFlow': df_no_defrosts['Flow'],
    'u_T_watrer_in': df_no_defrosts['Water_in'],
    'u_T_air': df_no_defrosts['T_air'],
    'x_areaLeakage': 5e-7,
    'x_areaSuctionValve': 5e-4,
    'x_areaDischargeValve': 5e-5,
    'x_relativeDeadSpace': 0.05,
    'x_driveEfficiency': 0.9
}


In [11]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

import numpy as np
from pymoo.core.problem import ElementwiseProblem

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=5,
                         n_obj=2,
                         n_ieq_constr=2,
                         xl=np.array([1e-8, 2e-6,1e-7 , 0.001, 0.85]),
                         xu=np.array([1e-6,1e-3,1e-4 ,0.1,0.98]))

    def _evaluate(self, x, out, *args, **kwargs):
        simulator = HeatPumpSimulator('../FMU/HPFMU_20_Linux.fmu')

        res = []

        # Evaluate the objective function using the simulator
        # n = len(INPUT['u_T_watrer_in'])
        # n = 5
        n = 5
        
        print("x :",x,"\n")

        for i in range(n):
            
            # xl=np.array([1e-8, 2e-6,1e-7 , 0.001, 0.85])
            # xu=np.array([1e-6,1e-3,1e-4 ,0.1,0.98])
            # x = np.random.uniform(low=xl, high=xu)

            start_values = {
                'u_compressorFrequency': INPUT['u_compressorFrequency'].iloc[i],
                'u_VFlow': INPUT['u_VFlow'].iloc[i],
                'u_T_watrer_in': INPUT['u_T_watrer_in'].iloc[i],
                'u_T_air': INPUT['u_T_air'].iloc[i],
                # Valeurs fixes des paramètres pour l'instant
                'x_areaLeakage': x[0],
                'x_areaSuctionValve': x[1],
                'x_areaDischargeValve': x[2],
                'x_relativeDeadSpace': x[3],
                'x_driveEfficiency': x[4]
            }
            finish_values = {
                'Water_out' : OUTPUT['Water_out'].iloc[i],
                'y_heatPower': OUTPUT['y_heatPower'].iloc[i],  
                'y_elecPower': OUTPUT['y_elecPower'].iloc[i],
                'y_COP': OUTPUT['y_COP'].iloc[i]
            }
            # print(start_values)
            differences = simulator.critere(start_values, finish_values)
            res.append(differences)

        out["F"] = res



### Verif fonctionnement de MyProblem

In [18]:
my_problem = MyProblem()

out_data = {}
resultats = my_problem._evaluate(x=[8.006259312689222e-07, 2.343953248846204e-05,6.224792405465264e-06, 0.08388855915929772, 0.97], out=out_data)

objectives = out_data["F"]

print("Objectives:", objectives)


x : [8.006259312689222e-07, 2.343953248846204e-05, 6.224792405465264e-06, 0.08388855915929772, 0.97] 

start_values:
u_compressorFrequency: 105.6
u_VFlow: 0.0004146853898097
u_T_watrer_in: 303.262817679558
u_T_air: 285.15
x_areaLeakage: 8.006259312689222e-07
x_areaSuctionValve: 2.343953248846204e-05
x_areaDischargeValve: 6.224792405465264e-06
x_relativeDeadSpace: 0.08388855915929772
x_driveEfficiency: 0.97

finish_values:
Water_out: 308.2487292817679
y_heatPower: 8628.77528413174
y_elecPower: 2022.29834254144
y_COP: 4.2796408839779
Le temps d'exécution dépasse 1 minutes. Tentative d'arrêt du processus.
start_values:
u_compressorFrequency: 98.4
u_VFlow: 0.0004090147329650083
u_T_watrer_in: 303.28900552486186
u_T_air: 285.15
x_areaLeakage: 8.006259312689222e-07
x_areaSuctionValve: 2.343953248846204e-05
x_areaDischargeValve: 6.224792405465264e-06
x_relativeDeadSpace: 0.08388855915929772
x_driveEfficiency: 0.97

finish_values:
Water_out: 308.3209392265193
y_heatPower: 8589.44430813188
y_el

### NSGA2

In [12]:
my_problem = MyProblem()

# Définir l'algorithme NSGA-II
algorithm = NSGA2(pop_size=5)

# Optimiser le problème en utilisant NSGA-II
result = minimize(my_problem,
                  algorithm,
                  ('n_gen', 5),
                  seed=1,
                  verbose=True)

x : [4.22851785e-07 7.20883844e-04 1.11426044e-07 3.09309247e-02
 8.69078266e-01] 

Le temps d'exécution dépasse 1 minutes. Tentative d'arrêt du processus.
Le temps d'exécution dépasse 1 minutes. Tentative d'arrêt du processus.
Le temps d'exécution dépasse 1 minutes. Tentative d'arrêt du processus.
Le temps d'exécution dépasse 1 minutes. Tentative d'arrêt du processus.
[WARNING] Warning: The following was detected at time: 12.21043418121939
[WARNING] TSMedia-Error(HPFMU.Evaporator.vleFluidCell[9].vleFluid.vleFluidPointer, HelmholtzModel::Trho(p,h), mediumPointerID=797): Could not find T and rho. p=1.26691e+06 h=-504822

[WARNING] The stack of functions is:
[WARNING] TSMedia.Internals.VLEFluidObjectFunctions.additionalProperties_phxi
[WARNING] TSMedia.Internals.VLEFluidObjectFunctions.additionalProperties_phxi(
Evaporator.vleFluidCell[9].vleFluid.p, 
Evaporator.vleFluidCell[9].vleFluid.h, 
Evaporator.vleFluidCell[9].vleFluid.xi, 
Evaporator.vleFluidCell[9].vleFluid.vleFluidPointer)
[WAR

: 

In [12]:

# Afficher les résultats
print("Optimal solutions:")
print(result.X)
print("Optimal objectives:")
print(result.F)


start_values:
u_compressorFrequency: 105.6
u_VFlow: 0.0004146853898097
u_T_watrer_in: 303.262817679558
u_T_air: 285.15
x_areaLeakage: 4.228517846555483e-07
x_areaSuctionValve: 0.0007208838444552738
x_areaDischargeValve: 1.1142604425275416e-07
x_relativeDeadSpace: 0.03093092469055214
x_driveEfficiency: 0.8690782658062247

finish_values:
Water_out: 308.2487292817679
y_heatPower: 8628.77528413174
y_elecPower: 2022.29834254144
y_COP: 4.2796408839779
Le temps d'exécution dépasse 1 minutes. Tentative d'arrêt du processus.
start_values:
u_compressorFrequency: 98.4
u_VFlow: 0.0004090147329650083
u_T_watrer_in: 303.28900552486186
u_T_air: 285.15
x_areaLeakage: 4.228517846555483e-07
x_areaSuctionValve: 0.0007208838444552738
x_areaDischargeValve: 1.1142604425275416e-07
x_relativeDeadSpace: 0.03093092469055214
x_driveEfficiency: 0.8690782658062247

finish_values:
Water_out: 308.3209392265193
y_heatPower: 8589.44430813188
y_elecPower: 2044.14917127072
y_COP: 4.214618784530389
Le temps d'exécution d

: 

## Autres NSGA

In [28]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

import numpy as np
from pymoo.core.problem import ElementwiseProblem

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=5,
                         n_obj=1,
                        #  n_obj=2,
                         n_ieq_constr=2,
                         xl=np.array([1e-8, 2e-6,1e-7 , 0.001, 0.85]),
                         xu=np.array([1e-6,1e-3,1e-4 ,0.1,0.98]))

    def _evaluate(self, x, out, *args, **kwargs):
        x_true = 2e-6
        f1 = x_true
        # f1 = 100 * (x[0]**2 + x[1]**2)
        f2 = (x[0]-1)**2 + x[1]**2

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8

        print("x :",x,"\n")

        # out["F"] = [f1, f2]
        # out["G"] = [g1, g2]

        out["F"] = [0.0]
        # out["G"] = [g1, g2]




# Instancier le problème
my_problem = MyProblem()

# Définir l'algorithme NSGA-II
algorithm = NSGA2(pop_size=10)

# Optimiser le problème en utilisant NSGA-II
result = minimize(my_problem,
                  algorithm,
                  ('n_gen', 10),
                  seed=1,
                  verbose=True)


x : [4.22851785e-07 7.20883844e-04 1.11426044e-07 3.09309247e-02
 8.69078266e-01] 

x : [1.01415209e-07 1.87887691e-04 3.46215166e-05 4.02799799e-02
 9.20046175e-01] 

x : [4.25002569e-07 6.85849061e-04 2.05247797e-05 8.79336262e-02
 8.53560387e-01] 

x : [6.73762835e-07 4.18470193e-04 5.59131139e-05 1.48983069e-02
 8.75753194e-01] 

x : [8.02737123e-07 9.68325053e-04 3.14110754e-05 6.95399390e-02
 9.63930590e-01] 

x : [8.95660597e-07 8.68741229e-05 4.00157284e-06 1.78132115e-02
 9.64158525e-01] 

x : [1.07363365e-07 4.22265410e-04 9.57931641e-05 5.37833632e-02
 9.39944025e-01] 

x : [3.22360475e-07 6.87127926e-04 8.34791046e-05 2.81053946e-03
 9.47518761e-01] 

x : [9.88972478e-07 7.48669323e-04 2.81163548e-05 7.91386535e-02
 8.63419381e-01] 

x : [4.53414591e-07 9.08778312e-04 2.94320534e-05 2.94897585e-02
 8.66903714e-01] 

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       10 |      1 |           INF |           INF |         

In [16]:
# Afficher les résultats
print("Optimal solutions:")
print(result.X)
print("Optimal objectives:")
print(result.F)

Optimal solutions:
None
Optimal objectives:
None


In [ ]:
def objective (x):
    res = 0

    # Evaluate the objective function using the simulator
    # n = len(INPUT['u_T_watrer_in'])
    # n = 5
    n = 5

    print("x :",x,"\n")

    for i in range(n):
        start_values = {
            'u_compressorFrequency': INPUT['u_compressorFrequency'].iloc[i],
            'u_VFlow': INPUT['u_VFlow'].iloc[i],
            'u_T_watrer_in': INPUT['u_T_watrer_in'].iloc[i],
            'u_T_air': INPUT['u_T_air'].iloc[i],
            # Valeurs fixes des paramètres pour l'instant
            'x_areaLeakage': x[0],
            'x_areaSuctionValve': x[1],
            'x_areaDischargeValve': x[2],
            'x_relativeDeadSpace': x[3],
            'x_driveEfficiency': x[4]
        }
        finish_values = {
            'Water_out' : OUTPUT['Water_out'].iloc[i],
            'y_heatPower': OUTPUT['y_heatPower'].iloc[i],  
            'y_elecPower': OUTPUT['y_elecPower'].iloc[i],
            'y_COP': OUTPUT['y_COP'].iloc[i]
        }
        # print(start_values)
        differences = simulator.critere(start_values, finish_values)

        # Ajouter differences à res uniquement si differences n'est pas -1
        if differences != -1:
            res += differences
    return res


In [27]:
import numpy as np
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.problems.functional import FunctionalProblem

# Définir la fonction objectif et les contraintes
objs = [
    lambda x: x[1] + x[4]
]




n_var = 5
# Définir le problème
problem = FunctionalProblem(n_var=5,objs = objs,
                         xl=np.array([1e-8, 2e-6,1e-7 , 0.001, 0.85]),
                         xu=np.array([1e-6,1e-3,1e-4 ,0.1,0.98]))

# Définir l'algorithme NSGA-II
algorithm = NSGA2(pop_size=10)

# Optimiser le problème
result = minimize(problem,
                  algorithm,
                  ('n_gen', 10),
                  seed=1,
                  verbose=True)

# Afficher les résultats
print("Optimal solutions:")
print(result.X)
print("Optimal objectives:")
print(result.F)


n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       10 |      1 |             - |             -
     2 |       20 |      2 |  0.000000E+00 |             f
     3 |       30 |      1 |  0.0002339776 |             f
     4 |       40 |      2 |  0.0002339776 |             f
     5 |       50 |      1 |  0.0003008262 |             f
     6 |       60 |      2 |  0.0003008262 |             f
     7 |       70 |      1 |  0.0029697862 |         ideal
     8 |       80 |      1 |  0.0001173061 |             f
     9 |       90 |      1 |  0.0004134294 |             f
    10 |      100 |      1 |  0.0004134294 |             f
Optimal solutions:
[4.52823166e-07 6.29408030e-04 7.83369500e-07 7.91414429e-02
 8.50233613e-01]
Optimal objectives:
[0.85086302]
